In [3]:
import os

In [4]:
%pwd

'c:\\Users\\deept\\ShopTalk\\research'

In [5]:
os.chdir("../")

In [6]:
%pwd

'c:\\Users\\deept\\ShopTalk'

In [7]:
from dataclasses import dataclass
from pathlib import Path


@dataclass(frozen=True)
class DataTransformationConfig:
    root_dir: Path
    data_dir: Path

In [8]:
from mlProject.constants import *
from mlProject.utils.common import read_yaml, create_directories

In [9]:
class ConfigurationManager:
    CONFIG_FILE_PATH = r'C:\Users\deept\ShopTalk\config\config.yaml'
    PARAMS_FILE_PATH = r"C:\Users\deept\ShopTalk\params.yaml"
    SCHEMA_FILE_PATH = r"C:\Users\deept\ShopTalk\schema.yaml"

    def __init__(
        self,
        config_filepath = Path(CONFIG_FILE_PATH),
        params_filepath = Path(PARAMS_FILE_PATH),
        schema_filepath = Path(SCHEMA_FILE_PATH)):

        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        self.schema = read_yaml(schema_filepath)
    
        create_directories([self.config.artifacts_root])

    
    def get_data_transformation_config(self) -> DataTransformationConfig:
        config = self.config.data_transformation
        create_directories([config.root_dir])

        data_transformation_config = DataTransformationConfig(
            root_dir=config.root_dir,
            data_dir=config.data_dir,
        )

        return data_transformation_config

In [10]:
import os
from mlProject import logger
from sklearn.model_selection import train_test_split
import pandas as pd

In [11]:
class DataTransformation:
    def __init__(self, config: DataTransformationConfig):
        self.config = config

    
    ## Note: You can add different data transformation techniques such as Scaler, PCA and all
    #You can perform all kinds of EDA in ML cycle here before passing this data to the model

    # I am only adding train_test_spliting cz this data is already cleaned up


    def train_test_spliting(self):
        try:
            # Base directory containing JSON files
            data_dir = Path(self.config.data_dir) 

            # Create Train and Test subfolders
            train_folder = data_dir / "Train"
            test_folder = data_dir / "Test"

            # Ensure Train and Test folders exist
            train_folder.mkdir(exist_ok=True)
            test_folder.mkdir(exist_ok=True)

            # Get list of all JSON files in data_dir
            json_files = [f for f in data_dir.iterdir() if f.suffix == ".json"]

            # Iterate through each JSON file
            for json_file in json_files:
                # Load data from JSON file
                data = pd.read_json(json_file, lines=True)
                
                # Split the data into training and test sets (75% train, 25% test)
                train, test = train_test_split(data, test_size=0.25, random_state=42)

                # Define file paths for train and test JSON files in respective folders
                train_file = train_folder / f"{json_file.stem}_train.json"
                test_file = test_folder / f"{json_file.stem}_test.json"

                # Save the train and test sets to JSON files
                train.to_json(train_file, orient='records', lines=True)
                test.to_json(test_file, orient='records', lines=True)

                # Logging information
                logger.info(f"Split {json_file.name} into train and test sets")
                logger.info(f"Training set shape: {train.shape}")
                logger.info(f"Test set shape: {test.shape}")

                # Print the shapes of the training and test sets
                print(f"Training set shape for {json_file.name}: {train.shape}")
                print(f"Test set shape for {json_file.name}: {test.shape}")

        except Exception as e:
            logger.error("Error during train/test split: ", exc_info=True)
            raise  # Re-raise the exception after logging
        

In [12]:
try:
    config = ConfigurationManager()
    data_transformation_config = config.get_data_transformation_config()
    data_transformation = DataTransformation(config=data_transformation_config)
    data_transformation.train_test_spliting()
except Exception as e:
    raise e

[2024-04-23 22:32:09,039: INFO: common: yaml file: C:\Users\deept\ShopTalk\config\config.yaml loaded successfully]
[2024-04-23 22:32:09,041: INFO: common: yaml file: C:\Users\deept\ShopTalk\params.yaml loaded successfully]
[2024-04-23 22:32:09,053: INFO: common: yaml file: C:\Users\deept\ShopTalk\schema.yaml loaded successfully]
[2024-04-23 22:32:09,055: INFO: common: created directory at: artifacts]
[2024-04-23 22:32:09,057: INFO: common: created directory at: artifacts/data_validation]
[2024-04-23 22:32:11,110: INFO: 911997196: Split listings_0.json into train and test sets]
[2024-04-23 22:32:11,111: INFO: 911997196: Training set shape: (6924, 28)]
[2024-04-23 22:32:11,112: INFO: 911997196: Test set shape: (2308, 28)]
Training set shape for listings_0.json: (6924, 28)
Test set shape for listings_0.json: (2308, 28)
[2024-04-23 22:32:13,584: INFO: 911997196: Split listings_1.json into train and test sets]
[2024-04-23 22:32:13,585: INFO: 911997196: Training set shape: (6924, 28)]
[2024-